In [1]:
import tensorflow as tf
from keras import backend as K
import os
import pandas as pd
import cv2
import numpy as np
import sys
sys.path.insert(0, "../")
from deeparuco.impl.losses import weighted_loss
from tensorflow.keras.models import load_model
from ultralytics import YOLO
import matplotlib.pyplot as plt
import csv
import time
import json

def custom_mae_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_pred - y_true), axis=-1)


2024-11-20 12:21:57.249713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-20 12:21:57.272165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-20 12:21:57.279314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 12:21:57.296026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-20 12:21:58.299534: W tensorflow/compiler/tf2

WARNING ⚠️ Python>=3.10 is required, but Python==3.9.5 is currently installed 


In [2]:
regressor = load_model("../models/nested2_regressor/corner_nested.keras",
    custom_objects={"weighted_loss": weighted_loss},)

I0000 00:00:1732126921.857175   73563 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-20 12:22:01.860118: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


One image

In [6]:
@tf.function(reduce_retracing=True)
def refine_corners(crops):
    return regressor(crops)

input_size = 128

norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-9)

pred_corner_batch = []
true_corner_batch = []

crop_dir  = '../dataset/nested2/crops/test/'
csv_path  = '../dataset/nested2/crops/test128.csv'
df = pd.read_csv(csv_path)

i = 155
filename = df.iloc[i][0]
file_id = os.path.splitext(filename)[0]
row = df.iloc[i]

# Read crop image
crop_path = crop_dir + filename
crops_ori = [cv2.imread(crop_path)]
crops = [norm(crop) for crop in crops_ori]
crop = crops[0]
crops_norm = [crop]

# Predict corners
norm_corners = refine_corners(np.array(crops_norm)).numpy()
norm_pred_corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]
norm_pred_corner = norm_pred_corners[0]
flat_norm_pred_corner = [item for t in norm_pred_corner for item in t]
pred_corner = [x * input_size for x in flat_norm_pred_corner]
pred_corner_batch.append(pred_corner)

# True corners
c1_x = float(row.iloc[1]) * input_size
c1_y = float(row.iloc[2]) * input_size
c2_x = float(row.iloc[3]) * input_size
c2_y = float(row.iloc[4]) * input_size
c3_x = float(row.iloc[5]) * input_size
c3_y = float(row.iloc[6]) * input_size
c4_x = float(row.iloc[7]) * input_size
c4_y = float(row.iloc[8]) * input_size
true_corner = [c1_x, c1_y, c2_x, c2_y, c3_x, c3_y, c4_x, c4_y]
true_corner_batch.append(true_corner)

im = cv2.imread(crop_path)

y_true = np.array(true_corner_batch)
y_pred = np.array(pred_corner_batch)
abs_diff = np.abs(y_true - y_pred)
mae = np.mean(abs_diff)
print(mae)

/tmp/ipykernel_6842/4102454369.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filename = df.iloc[i][0]


0.8186212833552782


Compute FPS

In [8]:
@tf.function(reduce_retracing=True)
def refine_corners(crops):
    return regressor(crops)

input_size = 128

norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-9)

crop_dir   = '../dataset/nested2/crops/test/'
label_dir  = '../dataset/nested2/labels/test/'

start_time = time.time()

for i in range(df.shape[0]):
    filename = df.iloc[i][0]
    file_id = os.path.splitext(filename)[0]
    row = df.iloc[i]

    # Read crop image
    crop_path = crop_dir + filename
    crops_ori = [cv2.imread(crop_path)]
    crops = [norm(crop) for crop in crops_ori]
    crop = crops[0]
    crops_norm = [crop]

    # Predict corners
    norm_corners = refine_corners(np.array(crops_norm)).numpy()

end_time = time.time()

/tmp/ipykernel_6842/3749531561.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filename = df.iloc[i][0]


In [11]:
(end_time - start_time) / 9000

0.00995179992251926

In [8]:
def write_to_csv(file_path, row_data, write_header=False):
    # Open the CSV file in append mode
    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writerow(row_data)

fieldnames = ['pic', 'tc1_x', 'tc1_y', 'tc2_x', 'tc2_y', 'tc3_x', 'tc3_y', 'tc4_x', 'tc4_y', 'pc1_x', 'pc1_y', 'pc2_x', 'pc2_y', 'pc3_x', 'pc3_y', 'pc4_x', 'pc4_y', 'location', 'reflection', 'mae']


@tf.function(reduce_retracing=True)
def refine_corners(crops):
    return regressor(crops)

input_size = 128

norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-9)

pred_corner_batch = []
true_corner_batch = []

crop_dir    = '../dataset/nested2/crops/test/'
csv_path    = '../dataset/nested2/crops/test128.csv'
result_path = '../dataset/nested2/corner_eval_model.csv'
detect_path = '../dataset/nested2/detect_eval_model_n.csv'

dfd = pd.read_csv(detect_path)
dfd = dfd.loc[dfd['TP'] == 1]
df = pd.read_csv(csv_path)

with open(result_path, mode='a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

for i in range(dfd.shape[0]):
    filename = dfd.iloc[i][0] + '.jpg'
    
    file_id = os.path.splitext(filename)[0]
    im_name = file_id.split('_')[0]
    row = df[df['pic'] == filename]


    # Read crop image
    crop_path = crop_dir + filename
    crops_ori = [cv2.imread(crop_path)]
    crops = [norm(crop) for crop in crops_ori]
    crop = crops[0]
    crops_norm = [crop]

    # Predict corners
    norm_corners = refine_corners(np.array(crops_norm)).numpy()
    norm_pred_corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]
    norm_pred_corner = norm_pred_corners[0]
    flat_norm_pred_corner = [item for t in norm_pred_corner for item in t]
    pred_corner = [x * input_size for x in flat_norm_pred_corner]
    pred_corner_batch.append(pred_corner)
    pred_corner = np.array(pred_corner)

    # True corners
    c1_x = float(row.iloc[0][1]) * input_size
    c1_y = float(row.iloc[0][2]) * input_size
    c2_x = float(row.iloc[0][3]) * input_size
    c2_y = float(row.iloc[0][4]) * input_size
    c3_x = float(row.iloc[0][5]) * input_size
    c3_y = float(row.iloc[0][6]) * input_size
    c4_x = float(row.iloc[0][7]) * input_size
    c4_y = float(row.iloc[0][8]) * input_size
    true_corner = [c1_x, c1_y, c2_x, c2_y, c3_x, c3_y, c4_x, c4_y]
    true_corner_batch.append(true_corner)
    true_corner = np.array(true_corner)

    # MAE
    mae = np.mean(np.abs(true_corner-pred_corner))

    row_data = { 'pic': filename,
    'tc1_x': true_corner[0], 'tc1_y': true_corner[1],
    'tc2_x': true_corner[2], 'tc2_y': true_corner[3],
    'tc3_x': true_corner[4], 'tc3_y': true_corner[5],
    'tc4_x': true_corner[6], 'tc4_y': true_corner[7],
    'pc1_x': pred_corner[0], 'pc1_y': pred_corner[1],
    'pc2_x': pred_corner[2], 'pc2_y': pred_corner[3],
    'pc3_x': pred_corner[4], 'pc3_y': pred_corner[5],
    'pc4_x': pred_corner[6], 'pc4_y': pred_corner[7], 'location': dfd.iloc[i][2],
    'reflection': row.iloc[0][11], 'mae': mae
    }
    write_to_csv(result_path, row_data)

/tmp/ipykernel_73563/2117450102.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filename = dfd.iloc[i][0] + '.jpg'
/tmp/ipykernel_73563/2117450102.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c1_x = float(row.iloc[0][1]) * input_size
/tmp/ipykernel_73563/2117450102.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c1_y = float(row.iloc[0][2]) * input_size
/tmp/ipykernel_73563/2117450102.py:61: FutureWarning: Series._

In [9]:
true_corner_batch = np.array(true_corner_batch)
pred_corner_batch = np.array(pred_corner_batch)
abs_diff = np.abs(true_corner_batch - pred_corner_batch)
mae = np.mean(abs_diff)
print(mae)

1.0730622116649415


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path_model = '../dataset/nested2/corner_eval_model.csv'

df = pd.read_csv(csv_path_model)
df_sub = df[df['location'] == 'inner']
true_corner_batch = []
pred_corner_batch = []

for i in range(len(df_sub)):
    tc1_x = df_sub.iloc[i][1]
    tc1_y = df_sub.iloc[i][2]
    tc2_x = df_sub.iloc[i][3]
    tc2_y = df_sub.iloc[i][4]
    tc3_x = df_sub.iloc[i][5]
    tc3_y = df_sub.iloc[i][6]
    tc4_x = df_sub.iloc[i][7]
    tc4_y = df_sub.iloc[i][8]
    true_corner = [tc1_x, tc1_y, tc2_x, tc2_y, tc3_x, tc3_y, tc4_x, tc4_y]
    true_corner_batch.append(true_corner)
    pc1_x = df_sub.iloc[i][9]
    pc1_y = df_sub.iloc[i][10]
    pc2_x = df_sub.iloc[i][11]
    pc2_y = df_sub.iloc[i][12]
    pc3_x = df_sub.iloc[i][13]
    pc3_y = df_sub.iloc[i][14]
    pc4_x = df_sub.iloc[i][15]
    pc4_y = df_sub.iloc[i][16]
    pred_corner = [pc1_x, pc1_y, pc2_x, pc2_y, pc3_x, pc3_y, pc4_x, pc4_y]
    pred_corner_batch.append(pred_corner)

true_corner_batch = np.array(true_corner_batch)
pred_corner_batch = np.array(pred_corner_batch)
abs_diff = np.abs(true_corner_batch - pred_corner_batch)
mae = np.mean(abs_diff)
print(mae)

/tmp/ipykernel_73563/1879305241.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tc1_x = df_sub.iloc[i][1]
/tmp/ipykernel_73563/1879305241.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tc1_y = df_sub.iloc[i][2]
/tmp/ipykernel_73563/1879305241.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tc2_x = df_sub.iloc[i][3]
/tmp/ipykernel_73563/1879305241.py:16: FutureWarning: Series.__getitem__ treating keys as positions is 

1.0704654264507703
